In [1]:
import logging
import coloredlogs
import requests
from time import sleep

from psycopg2.extensions import cursor
from psycopg2.extensions import connection
import pandas as pd

In [2]:
logging.basicConfig()
# logger = logging.getLogger("poll-ooni")
# logger.setLevel(logging.DEBUG)
coloredlogs.install()
coloredlogs.install(level='DEBUG')
# coloredlogs.install(level='INFO')

In [3]:
import config
from imp import reload
reload(config)
from config import config
import psycopg2

# connect to the db
my_connection = psycopg2.connect(**config['postgres'])
my_cursor = my_connection.cursor()
logging.info('Connected to database.')

2021-06-01 17:41:02 congratulations root[15547] INFO Connected to database.


# Testing inputs

In [5]:
import src.ooni.utils as utils

In [7]:
maybe_t = utils.get_latest_reading_time(my_cursor)
maybe_t

datetime.datetime(2021, 6, 1, 17, 30, 25, 644979, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-420, name=None))

In [8]:
ms = utils.query_measurements_after(maybe_t)

2021-06-01 17:41:33 congratulations urllib3.connectionpool[15547] DEBUG Starting new HTTPS connection (1): api.ooni.io:443
2021-06-01 17:41:34 congratulations urllib3.connectionpool[15547] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since=2021-06-02T00:30:25 HTTP/1.1" 200 90217


In [13]:
reload(utils)
for i, m in enumerate(ms):
    print(i, end=' ')
    utils.ingest_api_measurement(config['postgres'], m)

2021-06-01 17:43:17 congratulations urllib3.connectionpool[15547] DEBUG Starting new HTTPS connection (1): publicsuffix.org:443
2021-06-01 17:43:17 congratulations urllib3.connectionpool[15547] DEBUG https://publicsuffix.org:443 "GET /list/public_suffix_list.dat HTTP/1.1" 200 None
2021-06-01 17:43:18 congratulations tldextract[15547] DEBUG computed TLD diff:
--- .tld_set_snapshot

+++ /usr/local/lib/python3.6/dist-packages/tldextract/.tld_set

@@ -5,24 +5,17 @@

 !city.sapporo.jp
 !city.sendai.jp
 !city.yokohama.jp
-!teledata.mz
 !www.ck
 *.bd
-*.bn
 *.ck
 *.er
-*.fj
 *.fk
-*.gu
 *.jm
 *.kawasaki.jp
-*.ke
 *.kh
 *.kitakyushu.jp
 *.kobe.jp
-*.kw
 *.mm
-*.mz
 *.nagoya.jp
 *.nom.br
 *.np
@@ -30,10 +23,7 @@

 *.sapporo.jp
 *.sch.uk
 *.sendai.jp
-*.ye
 *.yokohama.jp
-*.zm
-*.zw
 0.bg
 1.bg
 2.bg
@@ -45,6 +35,7 @@

 7.bg
 8.bg
 9.bg
+9guacu.br
 a.bg
 a.se
 aa.no
@@ -59,6 +50,7 @@

 abbott
 abbvie
 abc
+abc.br
 abeno.osaka.jp
 abiko.chiba.jp
 abira.hokkaido.jp
@@ -77,25 +69,29 @@

 ac.cn
 ac.

2021-06-01 17:43:18 congratulations tldextract[15547] WARNING unable to cache TLDs in file /usr/local/lib/python3.6/dist-packages/tldextract/.tld_set: [Errno 13] Permission denied: '/usr/local/lib/python3.6/dist-packages/tldextract/.tld_set'


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 

AssertionError: 

In [17]:
ms[16]

{'anomaly': True,
 'confirmed': False,
 'failure': False,
 'input': 'http://www.xfocus.org/',
 'measurement_start_time': '2021-06-02T00:40:22Z',
 'measurement_url': 'https://ams-pg.ooni.org/api/v1/raw_measurement?report_id=20210602T000035Z_webconnectivity_RU_42668_n1_1RcriZTNgJWasodb&input=http%3A%2F%2Fwww.xfocus.org%2F',
 'probe_asn': 'AS42668',
 'probe_cc': 'RU',
 'report_id': '20210602T000035Z_webconnectivity_RU_42668_n1_1RcriZTNgJWasodb',
 'scores': {'analysis': {'blocking_type': 'http-failure'},
  'blocking_country': 0.0,
  'blocking_general': 1.0,
  'blocking_global': 0.0,
  'blocking_isp': 0.0,
  'blocking_local': 0.0},
 'test_name': 'web_connectivity'}

In [23]:
reload(utils)
hn = utils.get_hostname('http://www.xfocus.org')
utils.lookup_ip(my_cursor, my_connection, hn)

AssertionError: 

In [16]:
utils.ingest_api_measurement(config['postgres'], ms[16])

AssertionError: 

In [21]:
def rollback():
    my_cursor.execute("ROLLBACK")
    my_connection.commit()

# Visualization!

just connect to database to run cells below.

In [6]:
import json

def load (f):
    return json.load(open(f, 'r'))

sampleData = load('../../visualizations/globe/data/sampleData.json')

import country
export country
value

e -> i


countries are CC2s (perfect)

# Get directed data from the db

In [7]:
# rollback()

import pandas as pd

def unique_pairs (target_attribute:str, confirmed_only=True) -> pd.DataFrame:
    '''
    These are unique inputs that are blocked.
    
    We don't want every blocking example - if a site appears once or more, that's enough.
    '''
    my_cursor.execute(f'''
    SELECT DISTINCT input_url, probe_alpha2, {target_attribute}
    from ooni_web_connectivity_test
    {"where confirmed=True" if confirmed_only else ""}
    ''')
    pairs = my_cursor.fetchall()
    return pd.DataFrame(pairs, columns=['input', 'e', 'i']).dropna()


In [22]:
rollback()
my_cursor.execute(f'''
    SELECT measurement_start_time
    from ooni_web_connectivity_test
    order by measurement_start_Time
''')
my_cursor.fetchone()

(datetime.datetime(2021, 5, 26, 0, 16, 33, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-420, name=None)),)

In [40]:
import geoip2.database

In [8]:
tlds = unique_pairs('input_ip_alpha2')

In [9]:
tlds

,input,e,i
0,http://www.newnownext.com/,IR,NL
1,http://ilga.org/,IR,NL
2,http://alarabiya24.com/,SA,DE
3,http://ipvnews.org/,RU,NL
4,http://www.pokerstars.com/,RU,DE
...,...,...,...
1566,http://www.cbsnews.com/,IR,US
1567,http://www.medinkur.ru/,RU,VG
1568,http://abc.go.com/,IR,DE
1569,http://khodorkovsky.ru/,RU,DE


In [10]:
def find_attacks (**kwargs) -> pd.DataFrame:
    # get data
    pairs_tld = unique_pairs('tld_jurisdiction_alpha2', **kwargs)
#     pairs_ip = unique_pairs('input_ip_alpha2', **kwargs)
    attacks = pairs_tld
#     attacks = pd.concat([pairs_tld, pairs_ip])
    return attacks

In [11]:
from typing import List

def find_unsafe_alpha2s (alpha2s: List[str]):
    '''
    Find alpha 2 country codes that are in our list, but not in the GIOJS sample dataset.
    '''
    cc2_in_their =  set(flatten([(s['e'], s['i']) for s in sampleData]))
    possibly_unsafe_cc2 = alpha2s.difference(cc2_in_their)
    return possibly_unsafe_cc2

In [12]:
def count_attacks (attacks_df: pd.DataFrame, from_alpha2: str, to_alpha2: str) -> int:
    return len(attacks_df[
        (attacks_df['e']==from_alpha2) & 
        (attacks_df['i']==to_alpha2)])

In [13]:
# rollback()
from funcy import flatten

def create_weights (magic_constant=100000, **kwargs) -> dict:
    '''
    magic_constant scales number of attacks for the viz.
    '''
    attacks = find_attacks(**kwargs)
    # find distinct lines of attack (unique attacker->attackee vectors) 
    lines_of_attack = set([(v[0], v[1]) for v in attacks[['e', 'i']].values])
    # find the CC2s that GioJS won't support
    observed_alpha2s =  set(flatten(attacks[['e', 'i']].values.tolist()))
    possibly_unsafe_cc2 = find_unsafe_alpha2s(observed_alpha2s)
    # construct weights
    weights = []
    for line in lines_of_attack:
        e, i = line
        if e not in possibly_unsafe_cc2 and i not in possibly_unsafe_cc2 and e != i:
            # count the number of attacks across each line
            num_attacks = count_attacks(attacks, e, i)
            weight = num_attacks*magic_constant
            weights.append({
                'e': e,
                'i': i,
                'v': weight
            })
    return weights

In [14]:
weights_confirmed = create_weights(confirmed_only=True)
weights_all = create_weights(confirmed_only=False)

In [15]:

weights = {
    'dataSetKeys': ['confirmed', 'all'],
    'initDataSetKey': 'confirmed',
    'confirmed': weights_confirmed,
    'all': weights_all
}

json.dump(weights, open('../../visualizations/globe/data/attacks.json', 'w'))

# Some basic questions

how many 'bullets' do we see going by?

In [58]:
pairs_tld = unique_pairs('tld_jurisdiction_alpha2', confirmed_only=True)
pairs_tld
# about 418?

,input,e,i
0,http://www.turkpress.co/,SA,CO
1,http://www.anonymsurfen.com/,SA,US
2,http://www.babylon-x.com/,SA,US
3,http://thenationalcampaign.org/,IR,US
4,http://www.ansar.org/,IR,US
...,...,...,...
716,http://www.centcom.mil/,IR,US
717,http://baskino.me/,RU,US
718,http://www.cbsnews.com/,IR,US
719,http://www.hustler.com/,IR,US


about 418/day?

by destination, who's receiving them?

In [38]:
pairs_ip.groupby('i').count().sort_values('e', ascending=False)

NameError: name 'pairs_ip' is not defined

maybe just a big datacenter there...? load balancer? unclear.
unclear what to make of input IP in general...